In [1]:
################################################
# Numerical Methods for Stochastic             #
# Differential Equations                       #
################################################
#-----------------------------------------------
#-----------------------------------------------
################################################
# Compute the price of a European put option   #
# using Fourier methods for option pricing.    #
################################################
#-----------------------------------------------

#import libraries
import time
import numpy as np
import math
from matplotlib import pyplot as plt
from math import exp
from numpy.lib.scimath  import sqrt
from scipy.integrate import quad
#-----------------------------------------------
import warnings
warnings.filterwarnings("ignore")
#-----------------------------------------------

# to plot the results in the notebook:
%matplotlib inline

In [7]:
#-----------------------------------------------

def H93_call_value(S0 ,K,T,r,kappa_v,theta_v,sigma_v,rho,v0):
    ''' Valuation of European call option in H93 model via 
    Fourier-based approach.
    Parameters
    ==========
    initial_value : float
        initial stock/index level
    strike : float
        strike price
    maturity : datetime object
        time-to-maturity (for t=0)
    short_rate : float
        constant risk-free short rate
    kappa_v : float
        mean-reversion factor
    theta_v : float
        long-run mean of variance
    sigma_v : float
        volatility of variance
    rho : float
        correlation between variance and stock/index level
    variance: float
        initial level of variance 
    Returns
    =======
    call_value: float
        present value of European call option
    '''
    int_value = quad(lambda u:
                     H93_int_func(u, S0, K, T, r, kappa_v,
                                  theta_v, sigma_v, rho, v0),
                     0, np.inf, limit=250)[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) /
                     np.pi * int_value)
    return call_value


def H93_put_value(S0 ,K,T,r,kappa_v,theta_v,sigma_v,rho,v0):
    ''' Valuation of European call option in Heston (1993) model via
    Fourier-based approach. '''
    
    # Tic
    t = time.time()
    
    call_value = H93_call_value(S0 ,K,T,r,kappa_v,theta_v,sigma_v,rho,v0)
    put_value = call_value + K * math.exp(-r * T) - S0
    
     # Toc
    elapsed = time.time() - t
    
    print("Put Option Price       : %.4f" % put_value)
    print("Run time               : %.4f" % elapsed)
    return put_value



def H93_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    ''' Valuation of European call option in H93 model via Lewis (2001)
    Fourier-based approach: integration function.
    Parameter definitions see function H93_call_value.'''
    char_func_value = H93_char_func(u - 1j * 0.5, T, r, kappa_v,
                                    theta_v, sigma_v, rho, v0)
    int_func_value = 1 / (u ** 2 + 0.25) \
        * (np.exp(1j * u * np.log(S0 / K)) * char_func_value).real
    return int_func_value


def H93_char_func(u, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    ''' Valuation of European call option in H93 model 
    Fourier-based approach: characteristic function.'''
    c1 = kappa_v * theta_v
    c2 = -np.sqrt((rho * sigma_v * u * 1j - kappa_v) ** 2 -
                  sigma_v ** 2 * (-u * 1j - u ** 2))
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2) \
        / (kappa_v - rho * sigma_v * u * 1j - c2)
    H1 = (r * u * 1j * T + (c1 / sigma_v ** 2) *
          ((kappa_v - rho * sigma_v * u * 1j + c2) * T -
          2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))))
    H2 = ((kappa_v - rho * sigma_v * u * 1j + c2) / sigma_v ** 2 *
          ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T))))
    char_func_value = np.exp(H1 + H2 * v0)
    return char_func_value

In [8]:
################################################
# Input data.                                  #
################################################

initial_value = 100
strike        = 120
maturity      = 1
short_rate    = 0
kappa_v       = 1
theta_v       = 0.09
sigma_v       = 1
rho           = -0.3
volatility    = 0.09

In [9]:
strike   = 80                              #strike price(i)

FFT_P_80 = H93_put_value(initial_value,strike,maturity,short_rate,kappa_v,theta_v,sigma_v,rho,volatility) 

Put Option Price       : 3.4714
Run time               : 0.0080


In [10]:
strike    = 100                            #strike price(ii)

FFT_P_100 = H93_put_value(initial_value,strike,maturity,short_rate,kappa_v,theta_v,sigma_v,rho,volatility) 

Put Option Price       : 9.7738
Run time               : 0.0080


In [11]:
strike    = 120                            #strike price(iii)

FFT_P_120 = H93_put_value(initial_value,strike,maturity,short_rate,kappa_v,theta_v,sigma_v,rho,volatility) 

Put Option Price       : 23.4946
Run time               : 0.0110


In [12]:
#-----------------------------------------------
################################################
# Compare these results in terms of accuracy   # 
# and computational times with the put option  # 
# prices determined by the Euler Monte-Carlo   # 
################################################
#-----------------------------------------------

# Put-Call parity function for obtaining the price of the
# Put option from the price of the Call option,
# P=K-S_0+C.
def PutCallParity(C,K):
    P = K - S + C
    return P

In [ ]:
%run ../MonteCarlo/EuropeanCallOption.ipynb

In [13]:
print("#####################################################################")
print("For strike price K=80 the European put option is priced accordingly.")
print("Using Euler Monte Carlo Method: %.4f" % PutCallParity(C_80,80))
print("Using Fourier Method          : %.4f" % FFT_P_80)
print("#####################################################################")
print("For strike price K=100 the European put option is priced accordingly.")
print("Using Euler Monte Carlo Method: %.4f" % PutCallParity(C_100,100))
print("Using Fourier Method          : %.4f" % FFT_P_100)
print("#####################################################################")
print("For strike price K=120 the European put option is priced accordingly.")
print("Using Euler Monte Carlo Method: %.4f" % PutCallParity(C_120,120))
print("Using Fourier Method          : %.4f" % FFT_P_120)
print("#####################################################################")

#####################################################################
For strike price K=80 the European put option is priced accordingly.
Using Euler Monte Carlo Method: 3.3842
Using Fourier Method          : 3.4714
#####################################################################
For strike price K=100 the European put option is priced accordingly.
Using Euler Monte Carlo Method: 10.1552
Using Fourier Method          : 9.7738
#####################################################################
For strike price K=120 the European put option is priced accordingly.
Using Euler Monte Carlo Method: 23.6952
Using Fourier Method          : 23.4946
#####################################################################
